In [16]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os, logging, warnings, requests
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import List, Optional, Tuple
import numpy as np
import pandas as pd
import yfinance as yf

warnings.filterwarnings("ignore", category=FutureWarning)
logging.basicConfig(level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s")
log = logging.getLogger("futures_breakout_tradeplan")

# ==========================
# CONFIG
# ==========================
@dataclass
class Config:
    tickers: List[str]
    start_date: str
    end_date: str
    cache_dir: str
    result_dir: str
    min_volume: int
    buffer_pct: float
    target_rr: float
    capital_per_trade: float
    portfolio_capital: float
    margin_multiple: int
    telegram_enable: bool
    telegram_token: str
    telegram_chat_id: str
    use_ge_instead_of_gt: bool

CFG = Config(
    tickers= ['360ONE.NS', '3MINDIA.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABLBL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ACMESOLAR.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'AEGISLOG.NS', 'AEGISVOPAK.NS', 'AFCONS.NS', 'AFFLE.NS', 'AGARWALEYE.NS', 'AIAENG.NS', 'AIIL.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'AKZOINDIA.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ARE&M.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATHERENERG.NS', 'ATUL.NS', 'AUBANK.NS', 'AUROPHARMA.NS', 'AWL.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJAJHFL.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'BASF.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBTC.NS', 'BDL.NS', 'BEL.NS', 'BEML.NS', 'BERGEPAINT.NS', 'BHARATFORG.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BHEL.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUEJET.NS', 'BLUESTARCO.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'BSE.NS', 'BSOFT.NS', 'CAMPUS.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CAPLIPOINT.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOICEIN.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIPLA.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CONCOR.NS', 'CONCORDBIO.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CRISIL.NS', 'CROMPTON.NS', 'CUB.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DBREALTY.NS', 'DCMSHRIRAM.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'DLF.NS', 'DMART.NS', 'DOMS.NS', 'DRREDDY.NS', 'ECLERX.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ENRIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'FACT.NS', 'FEDERALBNK.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FIRSTCRY.NS', 'FIVESTAR.NS', 'FLUOROCHEM.NS', 'FORCEMOT.NS', 'FORTIS.NS', 'FSL.NS', 'GAIL.NS', 'GESHIP.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GMDCLTD.NS', 'GMRAIRPORT.NS', 'GODFRYPHLP.NS', 'GODIGIT.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GPIL.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GRSE.NS', 'GSPL.NS', 'GUJGASLTD.NS', 'GVT&D.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEG.NS', 'HEROMOTOCO.NS', 'HEXT.NS', 'HFCL.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HSCL.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDEA.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'IFCI.NS', 'IGIL.NS', 'IGL.NS', 'IIFL.NS', 'IKS.NS', 'INDGN.NS', 'INDHOTEL.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIGO.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFY.NS', 'INOXINDIA.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'IOB.NS', 'IOC.NS', 'IPCALAB.NS', 'IRB.NS', 'IRCON.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'ITC.NS', 'ITCHOTELS.NS', 'ITI.NS', 'J&KBANK.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JKCEMENT.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JPPOWER.NS', 'JSL.NS', 'JSWENERGY.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KAJARIACER.NS', 'KALYANKJIL.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KEI.NS', 'KFINTECH.NS', 'KIMS.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KPIL.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'KSB.NS', 'LALPATHLAB.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICHSGFIN.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LT.NS', 'LTF.NS', 'LTFOODS.NS', 'LTIM.NS', 'LTTS.NS', 'LUPIN.NS', 'M&M.NS', 'M&MFIN.NS', 'MAHABANK.NS', 'MAHSCOOTER.NS', 'MAHSEAMLES.NS', 'MANAPPURAM.NS', 'MANKIND.NS', 'MANYAVAR.NS', 'MAPMYINDIA.NS', 'MARICO.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MCX.NS', 'MEDANTA.NS', 'METROPOLIS.NS', 'MFSL.NS', 'MGL.NS', 'MINDACORP.NS', 'MMTC.NS', 'MOTHERSON.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MRF.NS', 'MRPL.NS', 'MSUMI.NS', 'MUTHOOTFIN.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NBCC.NS', 'NCC.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIVABUPA.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPC.NS', 'NTPCGREEN.NS', 'NUVAMA.NS', 'NUVOCO.NS', 'NYKAA.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'ONESOURCE.NS', 'ONGC.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PAYTM.NS', 'PCBL.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PGHH.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PNB.NS', 'PNBHOUSING.NS', 'POLICYBZR.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POONAWALLA.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'PPLPHARMA.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PTCIL.NS', 'PVRINOX.NS', 'RADICO.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RAMCOCEM.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RELINFRA.NS', 'RHIM.NS', 'RITES.NS', 'RKFORGE.NS', 'RPOWER.NS', 'RRKABEL.NS', 'RVNL.NS', 'SAGILITY.NS', 'SAIL.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'SRF.NS', 'STARHEALTH.NS', 'SUMICHEM.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TARIL.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TBOTEK.NS', 'TCS.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'THELEELA.NS', 'THERMAX.NS', 'TIINDIA.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TTML.NS', 'TVSMOTOR.NS', 'UBL.NS', 'UCOBANK.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UNITDSPR.NS', 'UNOMINDA.NS', 'UPL.NS', 'USHAMART.NS', 'UTIAMC.NS', 'VBL.NS', 'VEDL.NS', 'VENTIVE.NS', 'VGUARD.NS', 'VIJAYA.NS', 'VMM.NS', 'VOLTAS.NS', 'VTL.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ZFCVINDIA.NS', 'ZYDUSLIFE.NS'],
    start_date="2015-01-01",
    end_date=(datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d"),
    cache_dir="cache",
    result_dir="results",
    min_volume=10_000,
    buffer_pct=0.002,          # 0.2% above high
    target_rr=2.0,             # 1:2 R:R
    capital_per_trade=0.02,    # 2% risk
    portfolio_capital=10_000,  # ₹10 lakh
    margin_multiple=5,         # 5x margin
    telegram_enable=False,
    telegram_token="",
    telegram_chat_id="",
    use_ge_instead_of_gt=False,
)

# ==========================
# UTILITIES
# ==========================
def normalize(df):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [c[0] for c in df.columns]
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)
    df.index = df.index.tz_localize(None).normalize()
    return df

def get_data(ticker: str):
    os.makedirs(CFG.cache_dir, exist_ok=True)
    cache_path = f"{CFG.cache_dir}/{ticker}.parquet"
    df = pd.DataFrame()
    if os.path.exists(cache_path):
        try:
            df = pd.read_parquet(cache_path)
        except Exception:
            pass
    if df.empty:
        df = yf.download(ticker, start=CFG.start_date, end=CFG.end_date,
                         progress=False, auto_adjust=False, multi_level_index=False)
        if df.empty: return df
        df.to_parquet(cache_path)
    df = normalize(df)
    return df.dropna(subset=["Open","High","Low","Close","Volume"])

def add_features(df):
    d = df.copy()
    week_start = d.index - pd.to_timedelta(d.index.weekday, unit="D")
    d["WeekStart"] = week_start.normalize()
    d["Weekly_Open"] = d.groupby("WeekStart")["Open"].transform("first")
    d["MonthKey"] = d.index.to_period("M")
    d["Monthly_Open"] = d.groupby("MonthKey")["Open"].transform("first")
    for p in (20,50,200):
        d[f"SMA_{p}"] = d["Close"].rolling(p, min_periods=p).mean()
    return d

def resolve_date(df, desired):
    desired = pd.Timestamp(desired.date())
    if desired in df.index: return desired
    prev = df.index[df.index <= desired]
    return prev.max() if len(prev) else None

def cmp(a,b): return a>=b if CFG.use_ge_instead_of_gt else a>b

# ==========================
# FILTERS
# ==========================
def passes(df, dt):
    i = df.index.get_loc(dt)
    if i<200 or i<7: return False
    row, prev, past = df.iloc[i], df.iloc[i-1], df.iloc[i-7:i]
    rng_ok = all(cmp(row["High"]-row["Low"], r)
                 for r in (past["High"]-past["Low"]).tolist())
    daily_ok = cmp(row["Close"], row["Open"]) and cmp(row["Close"], prev["Close"])
    wk_ok = cmp(row["Close"], row["Weekly_Open"])
    mo_ok = cmp(row["Close"], row["Monthly_Open"])
    vol_ok = prev["Volume"] >= CFG.min_volume
    sma_ok = cmp(row["SMA_20"], row["SMA_50"]) and cmp(row["SMA_50"], row["SMA_200"])
    return all([rng_ok, daily_ok, wk_ok, mo_ok, vol_ok, sma_ok])

# ==========================
# TRADE PLAN + MARGIN LOGIC
# ==========================
def trade_plan(df, dt, ticker):
    row = df.loc[dt]
    entry = row["High"] * (1 + CFG.buffer_pct)
    stop = row["Low"]
    risk_per_share = entry - stop
    target = entry + CFG.target_rr * risk_per_share
    rr = CFG.target_rr

    # portfolio details
    total_cap = CFG.portfolio_capital
    max_risk_amt = total_cap * CFG.capital_per_trade
    raw_qty = max_risk_amt / risk_per_share
    max_qty_margin = (total_cap * CFG.margin_multiple) / entry
    qty = min(raw_qty, max_qty_margin)

    investment = qty * entry
    exposure_pct = (investment / (total_cap * CFG.margin_multiple)) * 100

    return {
        "Ticker": ticker,
        "Date": str(dt.date()),
        "Close": round(row["Close"],2),
        "Entry": round(entry,2),
        "Stop": round(stop,2),
        "Target": round(target,2),
        "Risk_per_share": round(risk_per_share,2),
        "RR": round(rr,2),
        "Qty": int(qty),
        "Investment": round(investment,0),
        "Exposure_%_of_margin": round(exposure_pct,2),
    }

def send_telegram(msg):
    if not CFG.telegram_enable: return
    if not (CFG.telegram_token and CFG.telegram_chat_id): return
    url = f"https://api.telegram.org/bot{CFG.telegram_token}/sendMessage"
    try:
        requests.post(url, data={"chat_id": CFG.telegram_chat_id, "text": msg})
    except Exception as e:
        log.warning(f"Telegram send failed: {e}")

# ==========================
# MAIN
# ==========================
def main(target_date=None):
    os.makedirs(CFG.result_dir, exist_ok=True)
    date_obj = datetime.strptime(target_date,"%Y-%m-%d") if target_date else datetime.now()
    date_obj = date_obj.replace(hour=0,minute=0,second=0,microsecond=0)
    log.info(f"Running breakout screener for {date_obj.date()}")
    trades=[]
    for t in CFG.tickers:
        df=get_data(t)
        if df.empty: continue
        df=add_features(df)
        dt=resolve_date(df,date_obj)
        if not dt: continue
        if passes(df,dt):
            plan=trade_plan(df,dt,t)
            trades.append(plan)
            log.info(f"✅ {t} | Entry={plan['Entry']} Stop={plan['Stop']} Qty={plan['Qty']} Investment={plan['Investment']}")
    if trades:
        dfp=pd.DataFrame(trades)
        out=f"{CFG.result_dir}/tradeplan_{date_obj.date()}.csv"
        dfp.to_csv(out,index=False)
        log.info(f"Saved {len(trades)} trade plans → {out}")
        if CFG.telegram_enable:
            msg = f"📈 {len(trades)} Trade Plans ({date_obj.date()}):\n"
            for x in trades:
                msg += f"{x['Ticker']} E:{x['Entry']} SL:{x['Stop']} Q:{x['Qty']}\n"
            send_telegram(msg)
    else:
        log.info("No signals today.")

if __name__=="__main__":
    TARGET_DATE=None   # None for today
    main(TARGET_DATE)


2025-10-16 14:16:23,509 | INFO | Running breakout screener for 2025-10-16
2025-10-16 14:16:35,651 | INFO | ✅ APARINDS.NS | Entry=8942.85 Stop=8335.0 Qty=0 Investment=2942.0
2025-10-16 14:16:38,148 | INFO | ✅ ASAHIINDIA.NS | Entry=952.9 Stop=899.55 Qty=3 Investment=3572.0
2025-10-16 14:16:49,138 | INFO | ✅ BHARATFORG.NS | Entry=1278.45 Stop=1230.0 Qty=4 Investment=5277.0
2025-10-16 14:17:42,674 | INFO | ✅ INDIANB.NS | Entry=807.56 Stop=764.35 Qty=4 Investment=3738.0
2025-10-16 14:17:52,657 | INFO | ✅ JSL.NS | Entry=810.22 Stop=779.65 Qty=6 Investment=5301.0
2025-10-16 14:18:18,157 | INFO | ✅ MOTHERSON.NS | Entry=108.77 Stop=104.69 Qty=49 Investment=5336.0
2025-10-16 14:18:23,594 | INFO | ✅ NESTLEIND.NS | Entry=1283.76 Stop=1216.5 Qty=2 Investment=3817.0
2025-10-16 14:18:50,799 | INFO | ✅ SAILIFE.NS | Entry=884.77 Stop=852.6 Qty=6 Investment=5501.0
2025-10-16 14:18:59,247 | INFO | ✅ SRF.NS | Entry=3212.21 Stop=3089.2 Qty=1 Investment=5223.0
2025-10-16 14:19:02,608 | INFO | ✅ SWIGGY.NS | 